# Keras and tensorflow probability on wine quality dataset

### aims of the notebook
Demonstrate tensorflow probability
(make this work)
taken from https://keras.io/examples/keras_recipes/bayesian_neural_networks/

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

Create training and evaluation datasets
Here, we load the wine_quality dataset using tfds.load(), and we convert the target feature to float. Then, we shuffle the dataset and split it into training and test sets. We take the first train_size examples as the train split, and the rest as the test split.



In [2]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

Compile, train, and evaluate the model

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

## Create model inputs

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

### Experiment 1: standard neural network
We create a standard deterministic neural network model as a baseline.


In [5]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

2023-02-14 15:48:11.312034: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Shuffling /Users/stephen.haddad/tensorflow_datasets/wine_quality/white/1.0.0.incompleteV36LK9/wine_quality-tra…

Dataset wine_quality downloaded and prepared to /Users/stephen.haddad/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


In [9]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 1s 9ms/step - loss: 30.3164 - root_mean_squared_error: 5.5060 - val_loss: 28.8226 - val_root_mean_squared_error: 5.3687
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 28.2571 - root_mean_squared_error: 5.3157 - val_loss: 27.1249 - val_root_mean_squared_error: 5.2082
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 26.5030 - root_mean_squared_error: 5.1481 - val_loss: 25.4013 - val_root_mean_squared_error: 5.0400
Epoch 4/100
17/17 [==============================] - 0s 4ms/step - loss: 24.8166 - root_mean_squared_error: 4.9816 - val_loss: 23.7276 - val_root_mean_squared_error: 4.8711
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: 23.1781 - root_mean_squared_error: 4.8144 - val_loss: 22.0871 - val_root_mean_squared_error: 4.6997
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: 21.5870 - root_mean_squared_error:

In [10]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")


Predicted: 6.0 - Actual: 6.0
Predicted: 5.3 - Actual: 5.0
Predicted: 6.6 - Actual: 8.0
Predicted: 5.4 - Actual: 6.0
Predicted: 6.5 - Actual: 7.0
Predicted: 5.2 - Actual: 4.0
Predicted: 6.4 - Actual: 7.0
Predicted: 5.5 - Actual: 5.0
Predicted: 6.4 - Actual: 6.0
Predicted: 5.7 - Actual: 5.0


In [18]:
list(test_dataset.unbatch())[0]

({'alcohol': <tf.Tensor: shape=(), dtype=float32, numpy=12.8>,
  'chlorides': <tf.Tensor: shape=(), dtype=float32, numpy=0.022>,
  'citric acid': <tf.Tensor: shape=(), dtype=float32, numpy=0.3>,
  'density': <tf.Tensor: shape=(), dtype=float32, numpy=0.98869>,
  'fixed acidity': <tf.Tensor: shape=(), dtype=float32, numpy=5.5>,
  'free sulfur dioxide': <tf.Tensor: shape=(), dtype=float32, numpy=20.0>,
  'pH': <tf.Tensor: shape=(), dtype=float32, numpy=3.34>,
  'residual sugar': <tf.Tensor: shape=(), dtype=float32, numpy=1.1>,
  'sulphates': <tf.Tensor: shape=(), dtype=float64, numpy=0.3799999952316284>,
  'total sulfur dioxide': <tf.Tensor: shape=(), dtype=float32, numpy=110.0>,
  'volatile acidity': <tf.Tensor: shape=(), dtype=float32, numpy=0.29>},
 <tf.Tensor: shape=(), dtype=float32, numpy=7.0>)

### Experiment 2: Bayesian neural network (BNN)
The object of the Bayesian approach for modeling neural networks is to capture the epistemic uncertainty, which is uncertainty about the model fitness, due to limited training data.

The idea is that, instead of learning specific weight (and bias) values in the neural network, the Bayesian approach learns weight distributions - from which we can sample to produce an output for a given input - to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights, and the training process is to learn the parameters of these distributions.



In [19]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

We use the tfp.layers.DenseVariational layer instead of the standard keras.layers.Dense layer in the neural network model.

In [22]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


The epistemic uncertainty can be reduced as we increase the size of the training data. That is, the more data the BNN model sees, the more it is certain about its estimates for the weights (distribution parameters). Let's test this behaviour by training the BNN model on a small subset of the training set, and then on the full training set, to compare the output variances.


In [23]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 298ms/step - loss: 36.5278 - root_mean_squared_error: 6.0425 - val_loss: 39.5646 - val_root_mean_squared_error: 6.2888
Epoch 2/500
5/5 [==============================] - 0s 10ms/step - loss: 36.1579 - root_mean_squared_error: 6.0121 - val_loss: 35.3394 - val_root_mean_squared_error: 5.9435
Epoch 3/500
5/5 [==============================] - 0s 10ms/step - loss: 37.3310 - root_mean_squared_error: 6.1088 - val_loss: 34.9572 - val_root_mean_squared_error: 5.9114
Epoch 4/500
5/5 [==============================] - 0s 10ms/step - loss: 37.3987 - root_mean_squared_error: 6.1143 - val_loss: 36.1607 - val_root_mean_squared_error: 6.0120
Epoch 5/500
5/5 [==============================] - 0s 10ms/step - loss: 37.0401 - root_mean_squared_error: 6.0848 - val_loss: 34.0106 - val_root_mean_squared_error: 5.8308
Epoch 6/500
5/5 [==============================] - 0s 10ms/step - loss: 37.4690 - root_mean_squared_error: 6.12

Since we have trained a BNN model, the model produces a different output each time we call it with the same input, since each time a new set of weights are sampled from the distributions to construct the network and produce an output. The less certain the mode weights are, the more variability (wider range) we will see in the outputs of the same inputs.



In [24]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.94, min: 5.5, max: 6.25, range: 0.76 - Actual: 6.0
Predictions mean: 5.53, min: 4.52, max: 5.97, range: 1.45 - Actual: 5.0
Predictions mean: 6.33, min: 5.98, max: 6.45, range: 0.47 - Actual: 8.0
Predictions mean: 5.44, min: 4.55, max: 5.87, range: 1.31 - Actual: 6.0
Predictions mean: 6.29, min: 5.97, max: 6.44, range: 0.47 - Actual: 7.0
Predictions mean: 5.21, min: 4.23, max: 5.97, range: 1.75 - Actual: 4.0
Predictions mean: 6.3, min: 5.98, max: 6.43, range: 0.46 - Actual: 7.0
Predictions mean: 5.48, min: 4.69, max: 5.96, range: 1.27 - Actual: 5.0
Predictions mean: 6.29, min: 6.06, max: 6.42, range: 0.36 - Actual: 6.0
Predictions mean: 5.58, min: 4.96, max: 5.98, range: 1.02 - Actual: 5.0


### Train BNN with the whole training set.

In [26]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 2s 31ms/step - loss: 36.0547 - root_mean_squared_error: 6.0042 - val_loss: 30.8611 - val_root_mean_squared_error: 5.5550
Epoch 2/500
17/17 [==============================] - 0s 4ms/step - loss: 34.7192 - root_mean_squared_error: 5.8919 - val_loss: 28.2056 - val_root_mean_squared_error: 5.3104
Epoch 3/500
17/17 [==============================] - 0s 4ms/step - loss: 31.1016 - root_mean_squared_error: 5.5765 - val_loss: 33.5175 - val_root_mean_squared_error: 5.7890
Epoch 4/500
17/17 [==============================] - 0s 4ms/step - loss: 30.4378 - root_mean_squared_error: 5.5166 - val_loss: 33.3387 - val_root_mean_squared_error: 5.7736
Epoch 5/500
17/17 [==============================] - 0s 4ms/step - loss: 28.7572 - root_mean_squared_error: 5.3622 - val_loss: 29.7361 - val_root_mean_squared_error: 5.4527
Epoch 6/500
17/17 [==============================] - 0s 4ms/step - loss: 29.4817 - root_mean_squared_error

Notice that the model trained with the full training dataset shows smaller range (uncertainty) in the prediction values for the same inputs, compared to the model trained with a subset of the training dataset.

### Experiment 3: probabilistic Bayesian neural network
So far, the output of the standard and the Bayesian NN models that we built is deterministic, that is, produces a point estimate as a prediction for a given example. We can create a probabilistic NN by letting the model output a distribution. In this case, the model captures the aleatoric uncertainty as well, which is due to irreducible noise in the data, or to the stochastic nature of the process generating the data.

In this example, we model the output as a IndependentNormal distribution, with learnable mean and variance parameters. If the task was classification, we would have used IndependentBernoulli with binary classes, and OneHotCategorical with multiple classes, to model distribution of the model output.

In [27]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

Since the output of the model is a distribution, rather than a point estimate, we use the negative loglikelihood as our loss function to compute how likely to see the true data (targets) from the estimated distribution produced by the model.

In [28]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)


Start training the model...
Epoch 1/1000
17/17 [==============================] - 2s 54ms/step - loss: 19.3865 - root_mean_squared_error: 6.7961 - val_loss: 16.1631 - val_root_mean_squared_error: 6.6297
Epoch 2/1000
17/17 [==============================] - 0s 4ms/step - loss: 19.0491 - root_mean_squared_error: 6.8005 - val_loss: 16.6362 - val_root_mean_squared_error: 6.8199
Epoch 3/1000
17/17 [==============================] - 0s 4ms/step - loss: 16.9090 - root_mean_squared_error: 6.5520 - val_loss: 12.1803 - val_root_mean_squared_error: 6.5755
Epoch 4/1000
17/17 [==============================] - 0s 4ms/step - loss: 11.4674 - root_mean_squared_error: 6.7585 - val_loss: 16.9146 - val_root_mean_squared_error: 6.3382
Epoch 5/1000
17/17 [==============================] - 0s 4ms/step - loss: 13.9891 - root_mean_squared_error: 6.6905 - val_loss: 9.4841 - val_root_mean_squared_error: 6.5373
Epoch 6/1000
17/17 [==============================] - 0s 4ms/step - loss: 11.1039 - root_mean_squared_

Now let's produce an output from the model given the test examples. The output is now a distribution, and we can use its mean and variance to compute the confidence intervals (CI) of the prediction.

In [29]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.82, stddev: 0.72, 95% CI: [7.23 - 4.41] - Actual: 6.0
Prediction mean: 5.67, stddev: 0.71, 95% CI: [7.07 - 4.28] - Actual: 5.0
Prediction mean: 6.56, stddev: 0.84, 95% CI: [8.2 - 4.92] - Actual: 8.0
Prediction mean: 5.72, stddev: 0.71, 95% CI: [7.11 - 4.33] - Actual: 6.0
Prediction mean: 6.56, stddev: 0.85, 95% CI: [8.23 - 4.89] - Actual: 7.0
Prediction mean: 5.44, stddev: 0.75, 95% CI: [6.9 - 3.98] - Actual: 4.0
Prediction mean: 6.56, stddev: 0.84, 95% CI: [8.21 - 4.91] - Actual: 7.0
Prediction mean: 5.6, stddev: 0.71, 95% CI: [6.99 - 4.21] - Actual: 5.0
Prediction mean: 6.51, stddev: 0.81, 95% CI: [8.1 - 4.91] - Actual: 6.0
Prediction mean: 5.43, stddev: 0.69, 95% CI: [6.79 - 4.07] - Actual: 5.0
